In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from nba import NbaTracker
from nba_api.stats.static.players import find_player_by_id
from nba_api.stats.static.teams import find_team_name_by_id
from genericpath import isdir
import tensorflow as tf
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import pandas as pd
import numpy as np
from nba_dataclasses import EventType, ResultClass
from sklearn.model_selection import train_test_split

nbaTracker = NbaTracker()
season_name = "2015-16"
nbaTracker.add_season(season_name)
season = nbaTracker.seasons[season_name]
season.load_possession_data()
games = season.games

train_game_ids, test_game_ids = train_test_split(
    list(games.keys()), test_size=0.1, random_state=432536)
len(train_game_ids), len(test_game_ids)


(1107, 123)

In [3]:
train_games = [games[game_id] for game_id in train_game_ids]
game = train_games[0]
for ge in game.game_events:
    if ge.event_type == EventType.PossessionTry:
        break

In [5]:
from nba_dataclasses import PossessionTry
result: PossessionTry = ge.result


In [7]:
result.num_fts

0

In [ ]:

columns = ['result_class', 'result_team', 'num_fts', 'shot_type',
           'is_blocked', 'is_putback', 'in_penalty', 'offense_is_home', 'score_margin']
non_player_columns = columns[:]
num_rebound_inputs = 6

catagorical_attributes = ['shot_type']
binary_attributes = ['is_blocked', 'is_putback',
                     'in_penalty', 'offense_is_home']
numerical_attributes = ['score_margin']

def get_try_df_from_games(games):
    tries = []
    for game in games:
        last_game_event = None
        for ge in game.game_events:
            if ge.event_type == EventType.PossessionTry:
                result_class = ge.result.result_class
                # if result_class == ResultClass.JUMPBALL:
                #     result_team = 2
                if result_class in {
                        ResultClass.OFF_REBOUND, ResultClass.FT, ResultClass.SAME_TEAM}:
                    result_team = 1
                else:
                    result_team = 0
                num_fts = ge.result.num_fts
                shot_type = ge.result.shot_type
                is_blocked = ge.result.is_blocked
                is_putback = last_game_event.is_putback
                in_penalty = ge.in_penalty
                offense_is_home = ge.offense_is_home
                score_margin = ge.score_margin

                rebound = [
                    result_class,
                    result_team,
                    num_fts,
                    shot_type,
                    is_blocked,
                    is_putback,
                    in_penalty,
                    offense_is_home,
                    score_margin,
                ]

                rebounds.append(rebound)
            last_game_event = ge
    return pd.DataFrame(rebounds, columns=columns)
